**Things im noticing that students keep getting errors from**

- Make sure your indentation matches the videos! Otherwise Python wont understand the code properly and it wont print out the summary of the model like in the video!


## **1) Importing Python Packages for GAN**


In [ ]:
from keras.datasets import mnist

from keras.models import Sequential
from keras.layers import BatchNormalization
from keras.layers import Dense, Reshape, Flatten
from keras.layers import LeakyReLU
from keras.optimizers import Adam

import numpy as np
!mkdir generated_images

## **2) Variables for Neural Networks & Data**

In [ ]:
img_width = 28
img_height = 28
channels = 1
img_shape = (img_width,img_height,channels)

latent_dim = 100
adam = Adam(lr=0.0001)
#adam = stochastic gradient descent algorithm

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


## **3) Building Generator**





In [ ]:
def build_generator():
  model = Sequential()
  #input layer
  #Dense layer is neural network layer that holds a bunch of parameters
  model.add(Dense(256, input_dim=latent_dim))
  #after every layer you need an activation function. 
  #relu is default activation function for NNs
  #this is a variation of relu called leakyrelu
  model.add(LeakyReLU(alpha=0.2))
  #This is the architecture that generates nice images. It makes the GAN more stable.
  model.add(BatchNormalization(momentum=0.8))

  #Stacking more layers, don't need input_dim after first insertion.
  model.add(Dense(512))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))

  model.add(Dense(1024))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))

  #output layer
  #Final activation layer is always tanh in GANS
  model.add(Dense(np.prod(img_shape), activation='tanh'))
  model.add(Reshape(img_shape))

  model.summary()
  return model

generator = build_generator()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               25856     
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 256)               0         
                                                                 
 batch_normalization (BatchN  (None, 256)              1024      
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 512)               131584    
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 512)               0         
                                                                 
 batch_normalization_1 (Batc  (None, 512)              2048      
 hNormalization)                                        

## **4) Building Discriminator**

In [ ]:
def build_discriminator():
  model = Sequential()

  model.add(Flatten(input_shape=img_shape))
  model.add(Dense(512))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Flatten(input_shape=img_shape))
  model.add(Dense(256))
  model.add(LeakyReLU(alpha=0.2))
  #Output layer
  model.add(Dense(1, activation='sigmoid'))
  model.summary()
  return model

discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense_4 (Dense)             (None, 512)               401920    
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 512)               0         
                                                                 
 flatten_1 (Flatten)         (None, 512)               0         
                                                                 
 dense_5 (Dense)             (None, 256)               131328    
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 256)               0         
                                                                 
 dense_6 (Dense)             (None, 1)                

## **5) Connecting Neural Networks to build GAN**

In [ ]:
GAN = Sequential()
discriminator.trainable = False
GAN.add(generator)
GAN.add(discriminator)


GAN.compile(loss='binary_crossentropy', optimizer=adam)
GAN.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 28, 28, 1)         1493520   
                                                                 
 sequential_1 (Sequential)   (None, 1)                 533505    
                                                                 
Total params: 2,027,025
Trainable params: 1,489,936
Non-trainable params: 537,089
_________________________________________________________________


## **6) Outputting Images**


In [ ]:
#@title
## **7) Outputting Images**
import matplotlib.pyplot as plt
import glob
import imageio
import PIL

save_name = 0.00000000

def save_imgs(epoch):
    #generate 25 images to fit on a 5 x 5 grid for our animation!
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, latent_dim))
    gen_imgs = generator.predict(noise)
    global save_name
    save_name += 0.00000001
    print("%.8f" % save_name)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            # axs[i,j].imshow(gen_imgs[cnt])
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("generated_images/%.8f.png" % save_name)
    print('saved')
    plt.close()

## **7) Training GAN**

In [ ]:
#make sure batch size is proportional to the amount of data (images) you have.
def train(epochs, batch_size=64, save_interval=200):
  (X_train, _), (_,_) = mnist.load_data()
  #print(X_train.shape)

  #normalizing data, use normalizer???
  X_train = X_train / 127.5 - 1.

  valid = np.ones((batch_size, 1))
  fakes = np.zeros((batch_size, 1))

  for epoch in range(epochs):
    idx = np.random.randint(0,X_train.shape[0], batch_size)
    imgs = X_train[idx]

    noise = np.random.normal(0,1, (batch_size, latent_dim))
    gen_imgs = generator.predict(noise)

    #Train the discriminator
    d_loss_real = discriminator.train_on_batch(imgs,valid)
    d_loss_fake = discriminator.train_on_batch(gen_imgs, fakes)
    #Taking average of the losses, use mean()?
    d_loss = np.add(d_loss_real, d_loss_fake) * 0.5

    #New noise to train gan
    noise = np.random.normal(0,1, (batch_size, latent_dim))

    #Train the GAN ALSO KNOWN AS INVERSE Y LABEL
    g_loss= GAN.train_on_batch(noise, valid)

    print("******* %d %d [D loss: %f, acc: %.2f%%] [G loss: %f]" % (epoch,j, d_loss[0], 100* d_loss[1], g_loss))
    if(epoch % save_interval) == 0:
      save_imgs(epoch)





train(30000, batch_size=64, save_interval=200)



2/2 [==============================] - 4s 5ms/step
************ 0 [D Loss: 0.825632, acc] 51.56 [G Loss: 0.573606]
1/1 [==============================] - 0s 106ms/step
0.00000001
saved
2/2 [==============================] - 0s 4ms/step
************ 1 [D Loss: 0.472811, acc] 85.16 [G Loss: 0.581127]
2/2 [==============================] - 0s 4ms/step
************ 2 [D Loss: 0.393633, acc] 78.91 [G Loss: 0.542758]
2/2 [==============================] - 0s 5ms/step
************ 3 [D Loss: 0.399443, acc] 64.06 [G Loss: 0.536899]
2/2 [==============================] - 0s 5ms/step
************ 4 [D Loss: 0.392746, acc] 64.06 [G Loss: 0.570945]
2/2 [==============================] - 0s 5ms/step
************ 5 [D Loss: 0.406358, acc] 61.72 [G Loss: 0.607248]
2/2 [==============================] - 0s 4ms/step
************ 6 [D Loss: 0.393188, acc] 64.84 [G Loss: 0.634438]
2/2 [==============================] - 0s 5ms/step
************ 7 [D Loss: 0.406518, acc] 64.84 [G Loss: 0.662637]
2/2 [=====

KeyboardInterrupt: ignored

### **8) Making GIF**

In [ ]:
anim_file = 'dcgan.gif'

with imageio.get_writer(anim_file, mode='I') as writer:
  filenames = glob.glob('generated_images/*.png')
  filenames = sorted(filenames)
  for filename in filenames:
    image = imageio.imread(filename)
    writer.append_data(image)
  image = imageio.imread(filename)
  writer.append_data(image)